# MySQL5.6

## dockerhostの設定

In [ ]:
%%bash
### mysqlクライアントのインストール(CentOS7)
sudo yum install -y http://dev.mysql.com/get/mysql-community-release-el7-5.noarch.rpm
sudo yum install -y mysql-community-client.x86_64

## コンテナの作成

In [1]:
%%bash
### 変数の設定
container="mysql5.6_centos7"
docker_compose_file="docker-compose_${container}.yml"
mysql_port="3306"
ssh_port="2222"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/mysql5.6_centos7:latest
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${mysql_port}:3306"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart mysqld.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d

### イメージ、コンテナの確認
docker images
docker ps -a

REPOSITORY                    TAG                      IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       virtual_for_jorurimail   35e0eebeeb79        17 hours ago        657MB
ftakao2007/mail_centos7       latest                   0733ee002c86        18 hours ago        656MB
ftakao2007/openldap_centos7   latest                   572b38993fa4        2 days ago          596MB
ftakao2007/mysql5.6_centos7   latest                   09a773ad8e34        4 days ago          2.16GB
ftakao2007/centos7            latest                   0fede72fa1fe        4 days ago          488MB
CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
REPOSITORY                    TAG                      IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       virtual_for_jorurimail   35e0eebeeb79        17 hours ago        657MB
ftakao2007/mail_centos7       latest                   0733ee002c86

Creating mysql5.6_centos7


In [3]:
%%bash
### mysql接続確認
mysql -ujoruri -p'pass' -h 127.0.0.1 -P 3306 -e "SELECT CURDATE();" 2> /dev/null

CURDATE()
2017-07-18


## VIPの付与

In [4]:
%%bash
### ネットワーク情報の確認
container="mysql5.6_centos7"
docker exec ${container} ifconfig -a 

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.2  netmask 255.255.0.0  broadcast 0.0.0.0
        ether 02:42:ac:11:00:02  txqueuelen 0  (Ethernet)
        RX packets 23  bytes 2020 (1.9 KiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 13  bytes 1090 (1.0 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1  (Local Loopback)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



In [5]:
%%bash
### VIPを付与する
container="mysql5.6_centos7"
ip="172.17.33.11"
netmask="255.255.0.0"
device="eth0"
docker exec ${container} ifconfig ${device}:1 ${ip} netmask ${netmask}

### ネットワーク確認
docker exec ${container} ifconfig -a

### MySQL接続確認
mysql -ujoruri -p'pass' -h ${ip} -P 3306 -e "SELECT CURDATE();" 2> /dev/null

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.2  netmask 255.255.0.0  broadcast 0.0.0.0
        ether 02:42:ac:11:00:02  txqueuelen 0  (Ethernet)
        RX packets 23  bytes 2020 (1.9 KiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 13  bytes 1090 (1.0 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

eth0:1: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.33.11  netmask 255.255.0.0  broadcast 172.17.255.255
        ether 02:42:ac:11:00:02  txqueuelen 0  (Ethernet)

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1  (Local Loopback)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

CURDATE()
2017-07-18
